# JACOBIAN

## Overview
The **Jacobian matrix** is a fundamental concept in multivariable calculus and mathematical optimization. It generalizes the derivative of a function to higher dimensions. For a vector-valued function $\mathbf{f}: \mathbb{R}^n \to \mathbb{R}^m$, the Jacobian matrix $J$ is an $m \times n$ matrix where each entry is the partial derivative of one component of $\mathbf{f}$ with respect to one variable:

The Jacobian matrix is given by $J = \left[ \frac{\partial f_i}{\partial x_j} \right]$, where $J_{ij} = \frac{\partial f_i}{\partial x_j}$.

Each row corresponds to the gradient of one output function with respect to all input variables. The Jacobian describes how small changes in the input variables affect the outputs. It is widely used in:
- Sensitivity analysis
- Nonlinear optimization
- Solving systems of nonlinear equations
- Machine learning (e.g., backpropagation)
- Robotics and control systems

**Example:**

Suppose $f(x, y) = x^2 y$. The Jacobian with respect to $x$ and $y$ is:

$\left[ \frac{\partial}{\partial x} (x^2 y) \quad \frac{\partial}{\partial y} (x^2 y) \right] = [2 x y \quad x^2]$

At $x=2$, $y=3$:
- $\frac{\partial}{\partial x} (x^2 y) = 2 \cdot 2 \cdot 3 = 12$
- $\frac{\partial}{\partial y} (x^2 y) = 2^2 = 4$

So the Jacobian matrix at this point is $[12, 4]$.

In this function, the Jacobian matrix is computed for a single mathematical expression with respect to specified variables using the CasADi library for symbolic differentiation and numerical evaluation.

## Usage
To use the `JACOBIAN` function in Excel, enter it as a formula in a cell, specifying your expression and variable values as described below:

```excel
=JACOBIAN(expression, var_values)
```
- `expression`: A string (e.g., "x**2 * y") representing the mathematical expression.
- `var_values`: A 2D array where the first row contains variable names and the second row contains their corresponding values. For example:

| x | y |
|---|---|
| 2 | 3 |

## Parameters
| Parameter      | Type         | Required | Description                                                                                       |
|----------------|--------------|----------|---------------------------------------------------------------------------------------------------|
| `expression`   | string       | Yes      | The mathematical expression as a string. For example, `"x**2 + y*z"`.                             |
| `var_values`   | list[list]   | Yes      | A list containing two inner lists: the first with variable names (strings), the second with their corresponding numerical values. Example: `[['x', 'y', 'z'], [1.0, 2.0, 3.0]]`. |

## Return Value
| Return Value    | Type                | Description                                                                                       |
|-----------------|---------------------|---------------------------------------------------------------------------------------------------|
| Jacobian Matrix | list[list[float]]   | On success, returns a 2D list of floats representing the Jacobian matrix.                         |
| Error Message   | string              | On failure, returns a string detailing the error encountered (e.g., invalid input, CasADi error).  |

## Benefits
- Automates the calculation of complex partial derivatives, saving time and reducing manual errors.
- Essential for advanced mathematical modeling, such as sensitivity analysis in financial models or gradient-based optimization algorithms.
- Leverages the robust symbolic differentiation capabilities of the CasADi library.
- Supports a single expression for flexibility in various analytical tasks.

## Examples

### Example 1: Single Expression
**Input:**
- expression: $"x**2 * y"$
- variables: $[[\'x\', \'y\'], [2.0, 3.0]]$

**Expected Output:**
$[[12.0, 4.0]]$

*Explanation:* The Jacobian of $"x**2 * y"$ with respect to $x$ is $2x y$, and with respect to $y$ is $x^2$. At $x=2.0, y=3.0$:
- Partial derivative w.r.t $x$: $2 \times 2.0 \times 3.0 = 12.0$
- Partial derivative w.r.t $y$: $2.0^2 = 4.0$

## Important Note on Mathematical Functions

When writing expressions, you must use CasADi's function names for mathematical operations instead of standard Python or NumPy names. For example, use $ca.sin(x)$ instead of $sin(x)$, and $ca.exp(x)$ instead of $exp(x)$.

Below is a table of common mathematical functions and their CasADi equivalents:

| Standard Name | CasADi Equivalent |
|--------------|-------------------|
| sin(x)       | ca.sin(x)         |
| cos(x)       | ca.cos(x)         |
| tan(x)       | ca.tan(x)         |
| exp(x)       | ca.exp(x)         |
| log(x)       | ca.log(x)         |
| sqrt(x)      | ca.sqrt(x)        |
| atan(x)      | ca.atan(x)        |
| asin(x)      | ca.asin(x)        |
| acos(x)      | ca.acos(x)        |
| sinh(x)      | ca.sinh(x)        |
| cosh(x)      | ca.cosh(x)        |
| tanh(x)      | ca.tanh(x)        |
| fabs(x)      | ca.fabs(x)        |
| floor(x)     | ca.floor(x)       |
| ceil(x)      | ca.ceil(x)        |

**Example:**
- Correct: $ca.sin(x) + y^3$
- Incorrect: $sin(x) + y^3$

If you use a standard function name instead of the CasADi equivalent, you will receive an error.

## Limitations
- The function returns specific error message strings if issues occur, such as:
  - The CasADi package not being found.
  - Invalid format or syntax in the $expression$.
  - Incorrectly structured $var\_values$ (e.g., mismatched number of variable names and their numerical values, incorrect data types).
  - Variables used in $expression$ that are not defined in $var\_values$.
  - Errors during CasADi's symbolic computation or numerical evaluation stages.

In [ ]:
import casadi as ca

def jacobian(expression, var_values):
    """
    Calculates the Jacobian matrix of a mathematical expression.

    Args:
        expression: A string representing the mathematical expression (e.g., "x**2 + y").
        var_values: A 2D list, where the first inner list contains the names of the variables
                    (e.g., [['x', 'y']]) and the second inner list contains their corresponding values
                    at which to evaluate the Jacobian (e.g., [[1.0, 2.0]]).

    Returns:
        A 2D list of floats representing the Jacobian matrix, or an error message string.
    """
    try:
        # Only accept string for expression
        if not isinstance(expression, str):
            return "expression must be a string."
        
        if not isinstance(var_values, list) or len(var_values) != 2:
            return "var_values should be a list of two lists: [[variable_names], [values]]"
        
        var_names = var_values[0]
        var_point_values = var_values[1]

        if not all(isinstance(name, str) for name in var_names):
             return "Variable names must be strings."
        if not all(isinstance(val, (int, float)) for val in var_point_values):
            return "Variable values must be numbers."
        if len(var_names) != len(var_point_values):
            return "Mismatch in the number of variable names and their values."

        # Create symbolic variables
        sym_vars = [ca.MX.sym(name) for name in var_names]
        
        # Create a dictionary for easy lookup of symbolic variables by name
        vars_dict = {name: sym_vars[i] for i, name in enumerate(var_names)}

        try:
            # Prepare a dictionary mapping variable names to CasADi symbols for eval
            eval_locals = vars_dict.copy()
            # Evaluate the string expression to a CasADi expression using eval with custom locals
            casadi_expr = eval(expression, {**globals(), **locals()}, eval_locals)
        except NameError as e:
            undefined_var = str(e).split("'")[1]
            if undefined_var not in var_names:
                return f"Mismatch between variables in expression and provided values. Undefined variable: {undefined_var}"
            return f"Invalid expression string. Error: {e}"
        except Exception as e:
            return f"Invalid expression string. Error: {e}"

        # Calculate the Jacobian
        J = ca.jacobian(casadi_expr, ca.vertcat(*sym_vars))

        # Create a CasADi function to evaluate the Jacobian
        # The order of sym_vars must match the order of var_point_values
        jacobian_func = ca.Function('jacobian_func', sym_vars, [J])

        # Evaluate the Jacobian at the given variable values
        result_matrix = jacobian_func(*var_point_values)
        
        # Convert CasADi DM object to a Python list of lists
        if isinstance(result_matrix, ca.DM):
            return result_matrix.full().tolist()
        else: # Should not happen if J is a matrix
            return "Error during CasADi calculation: Unexpected result type."

    except Exception as e:
        return str(e)

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_jacobian_single_expression():
    expression = "x**2 * y"
    variables = [['x', 'y'], [2.0, 3.0]]
    result = jacobian(expression, variables)
    assert isinstance(result, list)
    assert len(result) == 1
    assert len(result[0]) == 2
    assert result[0][0] == 12.0
    assert result[0][1] == 4.0

ipytest.run()

In [ ]:
# Interactive Demo
import gradio as gr

examples = [
    [
        "x**2 * y",
        [['x', 'y'], [2.0, 3.0]]
    ],
    [
        "ca.sin(x) + y**3",
        [['x', 'y'], [1.0, 2.0]]
    ],
    [
        "ca.exp(x*y) + x",
        [['x', 'y'], [0.5, 1.5]]
    ]
]

demo = gr.Interface(
    fn=jacobian,
    inputs=[
        gr.Textbox(label="Expression", value="x**2 * y"),
        gr.Dataframe(headers=["Variable Names", "Values"], label="Variable Names and Values", row_count=2, col_count=2, type="array", value=[['x', 'y'], [2.0, 3.0]])
    ],
    outputs=gr.Dataframe(label="Jacobian Matrix"),
    examples=examples,
    description="Calculate the Jacobian matrix of a mathematical expression with respect to specified variables.",
    flagging_mode="never",
)
demo.launch()